In [147]:
#!/usr/bin/python
import pickle
import os
import subprocess
import time
from operator import itemgetter
from scipy.stats import randint

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.naive_bayes import GaussianNB
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

import pandas as pd
from pandas.io import sql
import psycopg2
import base64
import numpy as np
import struct
from socket import inet_ntoa
import datetime

SIZE_OF_HEADER = 24
SIZE_OF_RECORD = 48

PROTOCOL = { 1: "ICMP", 6: "TCP", 14: "Telnet", 17: "UDP" }
def SperfTest1(data):
    times = []
    #for i in 
    for i in range(len(data)):
        if i == 0 or i == range(len(data)):
            continue        
        start1 = time.time()
        df = storage.filter_data_school(data.ix[i:i + 1], 'all')
        start2 = time.time()
        times.append(1)
        times[i - 1]= start2 - start1
    #print res
    return times

def SperfTest2(data):
    times = []
    #for i in 
    for i in range(len(data)):
        if i == 0 or i == range(len(data)):
            continue        
        times.append(1)
        start1 = time.time()
        df = storage.filter_data_school(data.ix[i:i + 1], 'd')
        df = storage.filter_data_school(data.ix[i:i + 1], 's_d')
        df = storage.filter_data_school(data.ix[i:i + 1], 'sp_d')
        df = storage.filter_data_school(data.ix[i:i + 1], 'sp_dp')
        start2 = time.time()        
        times[i - 1] += start2 - start1        
    #print res
    return times

def perfTest1(data):
    times = []
    #for i in 
    for i in range(len(data)):
        if i == 0 or i == range(len(data)):
            continue
        start1 = time.time()
        df = storage.filter_data_rkv(data.ix[i:i + 1], 'all')
        start2 = time.time()
        times.append(1)
        times[i - 1]= start2 - start1
    #print res
    return times

def perfTest2(data):
    times = []
    #for i in 
    for i in range(len(data)):
        if i == 0 or i == range(len(data)):
            continue
        times.append(1)
        start1 = time.time()
        df = storage.filter_data_rkv(data.ix[i:i + 1], 'd')
        df = storage.filter_data_rkv(data.ix[i:i + 1], 's_d')
        df = storage.filter_data_rkv(data.ix[i:i + 1], 'sp_d')
        df = storage.filter_data_rkv(data.ix[i:i + 1], 'sp_dp')
        start2 = time.time()        
        times[i - 1] += start2 - start1        
    #print res
    return times

def openIPList():
    with open('ip.dat', 'rd') as f:
        ipassoc = pickle.load(f)
    return ipassoc
    
def makeItFloat(iplist, ip):
    a = ip
    
    inlist = False
    for i in range(len(iplist)):
        if ip == iplist[i][0]:
            a = iplist[i][1]
            inlist = True
        
    if inlist == False:
        for i in range(len(iplist)):
            if ip == iplist[i][1]:                
                inlist = True        
    if inlist == False:
        a = '0'
    
    return a

class Storage():
    def __init__(self, config):
        self.conn = psycopg2.connect(host=config["hostname"], 
                                    user=config["username"], 
                                    password=config["password"], 
                                    database=config["database"])

    def close(self):
        self.conn.close()

    def get_upair_group(self, flow, t=None):
        if t == None or t == "all":
            return "all"
        s = ""
        for x in t.split('_'):
            if 'd' in x:
                s += "_"
            s += flow[x[0] + "addr"]
            if 'p' in x: 
                s += ":" + str(flow[x[0] + "port"])
        return s


    def nfdata_new(self):
        nfdata = {}
    
        nfdata['flow_count'] = 0
        nfdata['pcount'] = 0
        nfdata['bcount'] = 0
        nfdata['upairs'] = set()
        nfdata['protocol'] = { 'TCP': 0, 'ICMP': 0, 'Telnet': 0, 'UDP': 0 }
    
        return nfdata
    

    def nfdata_add(self, nfdata, flow):
        
        nfdata['pcount'] += flow['pcount']
        nfdata['bcount'] += flow['bcount']
        
        nfdata['upairs'] = nfdata['upairs'].union(flow['upairs'])
    
        nfdata['flow_count'] += 1
    
        for p in flow['protocol']:
            nfdata['protocol'][p] += flow['protocol'][p]
    
        return nfdata
    

    def select(self, t1=None, lim=None):
        limit = ""
        if t1 != None:
            limit += " WHERE time <= '%s' ORDER BY time DESC " % t1
        if lim != None:
            limit += " LIMIT %s" % lim
            
        sqlquery = "SELECT * FROM flows " + limit
        data = sql.read_sql(sqlquery, self.conn, index_col='id')
        return data

        
    def filter_data_school(self, data, nf_group_type=None):
        result = []
        for i in xrange(len(data)):
            nfc_group = { }
            
            dat = list((data.iloc[i]["data"]))
            #X-threads:
            for j in range(len(dat)):
                
                #1-thread:
                s_buf = dat[j]
                if s_buf == "":
                    continue
                buf = s_buf.decode('base64')
                flow_count = struct.unpack('B', buf[3:4])[0]
            
                for index in xrange(flow_count):
                    
                    offset = SIZE_OF_HEADER + (index * SIZE_OF_RECORD)
                    flow = { }
                
                    if len(buf) - offset > 47:
                        d = struct.unpack('!IIIIHH',buf[offset + 16:offset + 36])
                        flow['saddr'] = inet_ntoa(buf[offset + 0:offset + 4])
                        flow['sport'] = d[4]
                        flow['daddr'] = inet_ntoa(buf[offset + 4:offset + 8])
                        flow['dport'] = d[5]
                        flow['pcount'] = d[0]
                        flow['bcount'] = d[1]

                        flow['protocol'] = { }
                        flow['protocol'][PROTOCOL[ord(buf[offset + 38])]] = 1
                        
                        flow['upairs'] = set()
                        
                        flow['upairs'].add((buf[offset + 0:offset + 4], d[4], buf[offset + 4:offset + 8], d[5]))
                        s = self.get_upair_group(flow, nf_group_type)
                        if s not in nfc_group:
                            nfc_group[s] = self.nfdata_new()                           
                        nfc_group[s] = self.nfdata_add(nfc_group[s], flow)
                        
            for k in nfc_group:
                nfc_group[k]['ucount'] = len(nfc_group[k]['upairs'])
                del nfc_group[k]['upairs']
                
                row = [data.iloc[i]['time']]

                ip = k.split('_')
                
                if len(ip) == 1:
                    ip.append('')
                    
                row += ip
                row += [nfc_group[k]['ucount'], 
                       nfc_group[k]['pcount'], 
                       nfc_group[k]['bcount']]
                row += nfc_group[k]['protocol'].values()
                row += [data.iloc[i]['atk_desc']]
                row += [data.iloc[i]['atk_name']]
                result.append(row)
        
        columns = ['time','saddr', 'daddr','ucount','pcount', 'bcount'] + storage.nfdata_new()['protocol'].keys() + ['desc', 'target']
        
        if nf_group_type == None or nf_group_type == 'all':
            columns = ['time','-','ucount','pcount', 'bcount'] + storage.nfdata_new()['protocol'].keys() + ['desc', 'target']
        return pd.DataFrame(result, columns=columns)
        

def visualize_tree(tree, feature_names, path):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("data/dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "data/dt.dot", "-o", path]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")
    


def saveData(data, pathname):
    with open(pathname, 'wb') as f:
        pickle.dump(data, f)
    return 0

def loadData(pathname):
    with open(pathname, 'rb') as f:
        return pickle.load(f)

In [148]:
CONFIG = {
    "username": 'postgres',
    "password": '123456',
    "hostname": 'localhost',
    "database": 'traffic',
}
storage = Storage(CONFIG)
data = storage.select(t1='2016-03-23 13:43:16.588000', lim=4)
storage.close()
data['atk_desc'] = ""
data['atk_name'] = "b"
data

time  \
id                              
7  2016-03-23 13:43:16.588000   
6  2016-03-23 13:43:11.585000   
5  2016-03-23 13:43:06.582000   
4  2016-03-23 13:43:01.580000   

                                                 data atk_desc atk_name  
id                                                                       
7   [AAUAHj6AAABW8nNAAFM4sAABVSkAAAAAwKgFS1JwNqUAA...                 b  
6   [AAUAET6AE39W8nM7AAv2gAAA8OYAAAAAwKgDX57/j8MAA...                 b  
5   [AAUAAT6AE4NW8nM2AEPm0AAA8DEAAAAAwKgDX1vppSwAA...                 b  
4   [AAUABj6AE4JW8nMxAAD6AAABU10AAAAAJXFzvcCoAxQAA...                 b

In [176]:
#filtering:
d = storage.filter_data_school(data, 'd')

t = 'daddr'
#predict:
#pred = models['d'].predict(d)
pred = [3,4,5]
malware = tuple(d[d.index.isin(pred)][t])


#filtering:
d1 = storage.filter_data_school(data, 'sp_dp')

#predict:
d1.loc[d1[t].str.startswith(tuple(d[d.index.isin(pred)][t]))]


pred = list(d1.index)

In [177]:
def predict(d, options=None):
    

time               saddr                 daddr  \
14 2016-03-23 13:43:16.588000  192.168.5.75:60639  37.140.104.131:33885   
15 2016-03-23 13:43:16.588000  192.168.3.30:52575    5.141.204.80:12503   
21 2016-03-23 13:43:16.588000  192.168.3.20:58659    37.54.56.196:32734   

    ucount  pcount  bcount  UDP  ICMP  Telnet  TCP desc target  
14       1       1      52    0     0       0    1           b  
15       1       1      48    0     0       0    1           b  
21       1       3     152    0     0       0    1           b

In [94]:
config = CONFIG
conn = psycopg2.connect(host=config["hostname"], 
                                    user=config["username"], 
                                    password=config["password"], 
                                    database=config["database"])
conn.close()

In [93]:
t1 = '2016-03-23 13:43:16.588000'
limit = ""
limit += "WHERE time <= '%s' ORDER BY time DESC LIMIT 3" % t1
#limit += " LIMIT %s" % lim

sqlquery = "SELECT * FROM flows " + limit
data = sql.read_sql(sqlquery, conn, index_col='id')
data

time  \
id                              
7  2016-03-23 13:43:16.588000   
6  2016-03-23 13:43:11.585000   
5  2016-03-23 13:43:06.582000   

                                                 data  
id                                                     
7   [AAUAHj6AAABW8nNAAFM4sAABVSkAAAAAwKgFS1JwNqUAA...  
6   [AAUAET6AE39W8nM7AAv2gAAA8OYAAAAAwKgDX57/j8MAA...  
5   [AAUAAT6AE4NW8nM2AEPm0AAA8DEAAAAAwKgDX1vppSwAA...

In [82]:
#storage.filter_data_school(data2, 'all')
data

time  \
id                                 
1     2016-03-23 13:42:46.573000   
2     2016-03-23 13:42:51.575000   
3     2016-03-23 13:42:56.577000   
4     2016-03-23 13:43:01.580000   
5     2016-03-23 13:43:06.582000   
6     2016-03-23 13:43:11.585000   
7     2016-03-23 13:43:16.588000   
8     2016-03-23 13:43:21.591000   
9     2016-03-23 13:43:26.591000   
10    2016-03-23 13:43:31.594000   
11    2016-03-23 13:43:36.596000   
12    2016-03-23 13:43:41.599000   
13    2016-03-23 13:43:46.601000   
14    2016-03-23 13:43:51.602000   
15    2016-03-23 13:43:56.604000   
16    2016-03-23 13:44:01.605000   
17    2016-03-23 13:44:06.607000   
18    2016-03-23 13:44:11.613000   
19    2016-03-23 13:44:16.615000   
20    2016-03-23 13:44:21.617000   
21    2016-03-23 13:44:26.620000   
22    2016-03-23 13:44:31.624000   
23    2016-03-23 13:44:36.625000   
24    2016-03-23 13:44:41.626000   
25    2016-03-23 13:44:46.626000   
26    2016-03-23 13:44:51.626000   
27    2016-03-23 13:44:56.628000   
28    2016-03-23 13:45:01.631000   
29    2016-03-23 13:45:06.631000   
30    2016-03-23 13:45:11.631000   
...                          ...   
16678 2016-03-24 12:53:59.681000   
16679 2016-03-24 12:54:04.683000   
16680 2016-03-24 12:54:09.683000   
16681 2016-03-24 12:54:14.685000   
16682 2016-03-24 12:54:19.687000   
16683 2016-03-24 12:54:24.687000   
16684 2016-03-24 12:54:29.688000   
16685 2016-03-24 12:54:34.688000   
16686 2016-03-24 12:54:39.688000   
16687 2016-03-24 12:54:44.690000   
16688 2016-03-24 12:54:49.692000   
16689 2016-03-24 12:54:54.693000   
16690 2016-03-24 12:54:59.693000   
16691 2016-03-24 12:55:04.694000   
16692 2016-03-24 12:55:09.696000   
16693 2016-03-24 12:55:14.698000   
16694 2016-03-24 12:55:19.698000   
16695 2016-03-24 12:55:24.700000   
16696 2016-03-24 12:55:29.703000   
16697 2016-03-24 12:55:34.703000   
16698 2016-03-24 12:55:39.705000   
16699 2016-03-24 12:55:44.707000   
16700 2016-03-24 12:55:49.707000   
16701 2016-03-24 12:55:54.709000   
16702 2016-03-24 12:55:59.711000   
16703 2016-03-24 12:56:04.713000   
16704 2016-03-24 12:56:09.715000   
16705 2016-03-24 12:56:14.717000   
16706 2016-03-24 12:56:19.717000   
16707 2016-03-24 12:56:24.719000   

                                                    data atk_desc atk_name  
id                                                                          
1      [AAUAAj6AE4RW8nMiAEPLeAAA7aQAAAAAwKgAZQgICAgAA...                 b  
2      [AAUAHj6AAABW8nMnAI2OSAAA7kYAAAAAwKgDRU6b1acAA...                 b  
3      [AAUABz6AE4RW8nMsAEPfAAAA7voAAAAAwKgDFCVxc70AA...                 b  
4      [AAUABj6AE4JW8nMxAAD6AAABU10AAAAAJXFzvcCoAxQAA...                 b  
5      [AAUAAT6AE4NW8nM2AEPm0AAA8DEAAAAAwKgDX1vppSwAA...                 b  
6      [AAUAET6AE39W8nM7AAv2gAAA8OYAAAAAwKgDX57/j8MAA...                 b  
7      [AAUAHj6AAABW8nNAAFM4sAABVSkAAAAAwKgFS1JwNqUAA...                 b  
8      [AAUAGj6AE3tW8nNFAA1Z+AAA8sEAAAAAwKgDWSX8qpIAA...                 b  
9      [AAUAHj6AAABW8nNKAFJtkAABVwEAAAAAwKgAZcCoCBoAA...                 b  
10     [AAUAAj6AE3lW8nNPAAERcAAA89cAAAAAXZ6Gd8CoAyUAA...                 b  
11     [AAUAHj6AB9NW8nNUANEPiAABXXQAAAAAwKgFS1OOb6YAA...                 b  
12     [AAUADT6AE3pW8nNZAADuSAABWYMAAAAAwKgAZcCoCBoAA...                 b  
13     [AAUAHj6AB9BW8nNeALBBGAABXr4AAAAAwKgFS7DX+DAAA...                 b  
14     [AAUAGj6AE3tW8nNjAAERcAAA9bYAAAAAwKgDWSX8qpIAA...                 b  
15     [AAUAHj6AAABW8nNoAFIIAAABWxkAAAAAwKgDX22ucIQAA...                 b  
16     [AAUACD6AE4dW8nNtAEP+QAABXGMAAAAATXmGqcCoBRYAA...                 b  
17     [AAUAHj6AB9NW8nNyAKjK8AABYpwAAAAAwKgDFFv0VDgAA...                 b  
18     [AAUAEz6AE3lW8nN3AAD96AAA+NkAAAAAwKgIZcCoA0UAA...                 b  
19     [AAUAHj6AAABW8nN8AJKjEAAA+eEAAAAAwKgIZcCoA0UAA...                 b  
20     [AAUAAj6AE35W8nOBAAENiAABYGsAAAAAwKgFS9U7ll8AA...                 b  
21     [AAUAAj6AE39W8nOGAAG1gAAA+5EAAAAAwKgFSwUsqDUAA...         